First I need to download the necessary libraries. 

In [1]:
# LIBRARIES
library(tidyverse)
library(tidymodels)
library(vroom)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

Now I need to read in the data.

In [2]:
# READ IN DATA
all_train <- vroom('/kaggle/input/allstate-claims-severity/train.csv')
all_test <- vroom('/kaggle/input/allstate-claims-severity/test.csv')

Rows: 188318 Columns: 132
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (116): cat1, cat2, cat3, cat4, cat5, cat6, cat7, cat8, cat9, cat10, cat1...
dbl  (16): id, cont1, cont2, cont3, cont4, cont5, cont6, cont7, cont8, cont9...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 125546 Columns: 131
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (116): cat1, cat2, cat3, cat4, cat5, cat6, cat7, cat8, cat9, cat10, cat1...
dbl  (15): id, cont1, cont2, cont3, cont4, cont5, cont6, cont7, cont8, cont9...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Now I need to perform some feature engineering. First I decided to take the log of the loss variable. Next I created my recipe, which has explanations included in the code. Then I chose my model and fit the workflow. Then I created my predictions and correctly formatted these predictions so they could be successfully submitted to Kaggle. Lastly, I created my csv file for submission.

In [3]:
# FEATURE ENGINEERING

# take log of loss variable
all_train$loss <- log(all_train$loss)

# recipe
my_recipe <- recipe(loss ~ ., all_train) %>% 
  update_role(id, new_role = 'ID') %>% # update id role
  step_scale(all_numeric_predictors()) %>% # normalize all numeric predictors to have sd = 1
  step_corr(all_numeric_predictors(), threshold = .6) %>% # remove all numeric predictors that have a correlation with other variables that is over 0.6
  step_novel(all_nominal_predictors()) %>% # assign new factor values a "new" value
  step_unknown(all_nominal_predictors()) %>% # assign missing values an "unknown" value
  step_dummy(all_nominal_predictors()) %>% # create dummy variables for all nominal predictors
  prep()

# model
my_mod <- linear_reg() %>% # Type of model
  set_engine("lm")

# workflow
lin_wf <- workflow() %>%
  add_recipe(my_recipe) %>%
  add_model(my_mod) %>%
  fit(data = all_train) # Fit the workflow

# predictions
lin_predictions <- predict(lin_wf,
                              new_data= all_test)

# correctly format predictions
lin_predictions <- lin_predictions %>%
  bind_cols(., all_test) %>%
  select(id, .pred) %>%
  rename(loss = .pred) %>%
  mutate(loss = exp(loss)) # must do this because we previously took the log of loss

# create csv file
vroom_write(x=lin_predictions, file="linear_predictions.csv", delim=",")

Warning message in predict.lm(object = object$fit, newdata = new_data, type = "response", :
“prediction from a rank-deficient fit may be misleading”
